In [1]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import time
import numpy as np
import openai

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener el valor de OPENAI_API_KEY
api_key = os.getenv("OPENAI_API_KEY")

gpt_client = OpenAI(
    api_key = api_key,
)

In [2]:
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    response = gpt_client.embeddings.create(input = [text], model=model).data[0].embedding
    return response

In [3]:
from chromadb import PersistentClient
client = PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="productos")

In [4]:
def generate_response(query, productos_relacionados):
    """
    Genera una respuesta basada en una consulta y productos relacionados.
    """
    response = f"Query: {query}\n\nRelated Products:\n"
    for producto in productos_relacionados:
        # Asegúrate de que las claves coincidan con las que se usaron en productos_relacionados
        response += (
            f"Producto: {producto.get('producto', 'N/A')}\n"
            f"Descripcion: {producto.get('producto', 'N/A')}\n"
            f"Precio: {producto.get('precio', 'N/A')}\n"
            f"Enlace: {producto.get('url', 'N/A')}\n"
            f"Market: {producto.get('market', 'N/A')}\n"
            f"Categorias: {producto.get('categorias', 'N/A')}\n\n"
        )
    return response



In [5]:
# prompt = '''
# Asumir que eres un profesional en la preparación de alimentos. Solo nos brindarás información sobre los productos y no sobre la preparación de los alimentos. No considerarás insumos para equipos de cocina y tampoco otros que no estén relacionados. Las bebidas de todo tipo pueden incluirse en la respuesta.
# '''

In [6]:
query_texto = "aceite de cocina"
query_embedding = get_embedding(query_texto)
print(query_embedding)

[-0.006830474361777306, -0.010189616121351719, 0.0008538092952221632, -0.02128996141254902, 0.0007791525567881763, -0.010585585609078407, -0.006107829976826906, -0.004170878790318966, 0.00366931757889688, -0.0281402338296175, 0.018887747079133987, 0.026754340156912804, -0.013225382193922997, -0.0014188907807692885, -0.03679876774549484, 0.025711622089147568, 0.019336512312293053, 0.01896694116294384, 0.03344622626900673, -0.023032227531075478, -0.011747095733880997, 0.02455011010169983, 0.007741203997284174, -0.028641795739531517, -0.014703668653964996, -0.00872452836483717, 0.003897000104188919, -0.024523712694644928, -0.004781331866979599, -0.022161094471812248, 0.03785468637943268, 0.006757880095392466, -0.012789815664291382, -0.0018214598530903459, -0.006540096830576658, 0.003337693167850375, -0.01467727031558752, -0.025645626708865166, 0.002461610594764352, 0.013205583207309246, 0.013740142807364464, -0.013489361852407455, -0.0010938658379018307, -0.0028658295050263405, -0.0147432

In [7]:
collection.peek(limit=1)

{'ids': ['b33135fe-6f8c-446f-a3e8-aa062ff06cba'],
 'embeddings': array([[ 0.00302135,  0.0149789 ,  0.01100438, ..., -0.01178116,
         -0.01896637,  0.00985863]]),
 'documents': ['Producto: Agua Tónica BRITVIC Paquete 12un Lata 150ml. Descripción: .Agua tónica Marca: BRITVIC  Contenido: 12un de 150ml  .AGUASplazaVea presenta una gran variedad de Bebidas para que elijas tu favorita de acuerdo a diferentes situaciones u ocasiones especiales. Hidratarse es muy importante para el correcto cuidado de nuestro organismo, además que nos ayuda a lucir mejor, porque una piel hidratada se refleja siempre en el exterior. Por tal motivo, te ofrecemos diferentes presentaciones de Aguas: agua en sixpack, agua en bidón, agua personal, agua de 2.5 litros y más.Puedes elegir entre Agua Mineral y Agua de Mesa en las marcas más populares del mercado. También puedes revisar nuestras opciones de Aguas Saborizadas para que te hidrates con una opción más rica pero libre de azúcares. . Categorías: Bebidas,

In [8]:
def query_products_chromadb(query_embedding, k=3):
    # Buscar en ChromaDB los más similares
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=["metadatas", "distances"]
    )

    # Formatear resultados
    # ChromaDB devuelve metadatas como una lista dentro del diccionario
    productos = [
        {
            "producto": meta["producto"],
            "descripcion": meta["descripcion"],
            "precio": meta["precio"],
            "market": meta["market"],
            "url": meta["url"],
            "categorias": meta["categorias"]
        }
        for meta in results["metadatas"][0]  # Accedemos usando la notación de diccionario
    ]

    return {
        "productos": productos,
        "distances": results["distances"][0] if "distances" in results else None
    }

In [9]:
productos_relacionados = query_products_chromadb(query_embedding = query_embedding, k=5)
print(productos_relacionados)
# respuesta_gpt = generate_response("¿qué productos puedo utilizar para freir papas?", productos_relacionados)
# print(respuesta_gpt)


{'productos': [{'producto': 'Aceite de Soya BASSO Botella 1L', 'descripcion': '.Aceite vegetal Ideal para preparar frituras  No absorve fácilmente  .ACEITEplazaVea cuenta con una amplia variedad de Abarrotes  para que puedas elegir el que más se adecua a tus necesidades. Dentro de los  Abarrotes encontramos al aceite , un producto muy usado en la cocina gracias a su versatilidad. Por ejemplo el Aceite de Oliva cuenta con un alto contenido en ácidos grasos monoinsaturados y poliinsaturados que permitirán aportar grandes beneficios en nuestro organismo como : Eleva los niveles de colesterol HDL (bueno), Disminuye el colesterol LDL-c (colesterol malo), Beneficia el control de la hipertensión arterial, entre otros. Es ideal para consumirlo de manera directa, ya sea en ensaladas o como complemento alimentario, de este modo podrás absorver al máximo sus nutrientes.Además también puedes optar por el Aceite Vegetal que gracias a su composición es perfecto para la preparación de cualquier recet

In [10]:
# # Formatear respuesta
# def format_response(api_response):
#     formatted = []
#     for doc in api_response['documents'][0]:
#         product_info = {}
#         lines = doc.split('.')
#         product_info['Producto'] = lines[0].replace("Producto:", "").strip()
#         product_info['Descripción'] = ".".join(lines[1:]).strip()
#         formatted.append(product_info)
#     return formatted
#
# formatted_response = format_response(productos_relacionados)
#
# import json
# print(json.dumps(formatted_response, indent=4, ensure_ascii=False))

In [11]:
def format_response_with_openai(api_response, client):
    # Obtenemos los productos del resultado de query_products_chromadb
    productos = api_response['productos']
    formatted_documents = []

    for producto in productos:
        # Creamos un texto estructurado con la información del producto
        producto_text = f"""
        Producto: {producto['producto']}
        Descripción: {producto['descripcion']}
        Precio: S/ {producto['precio']}
        Market: {producto['market']}
        Categorías: {producto['categorias']}
        URL: {producto['url']}
        """

        # Prompt para generar una respuesta natural
        prompt = f"""
        El siguiente es un producto con su descripción detallada. Transforma esta información en un texto natural y amigable para el usuario,
        mencionando sus características principales y precio. No añadas texto de vendedor, sino texto de información útil al consumidor. Los productos sugeridos son para consumo humano, si detectas que alguno no lo es, no lo incluyas en tu respuesta. Retira las frases que no sean útiles para el usuario. Retira las frases correspondientes al marketing.

        {producto_text}

        Responde de forma clara y profesional, como si estuvieras recomendando el producto a un cliente.
        """

        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Eres un asistente especializado en redacción clara y profesional, experto en productos de supermercado."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300
            )
            # Extrae el contenido generado
            generated_text = response.choices[0].message.content.strip()
            formatted_documents.append(generated_text)
        except Exception as e:
            formatted_documents.append(f"Error procesando el documento: {str(e)}")

    return formatted_documents

# Ejemplo de uso:
productos_relacionados = query_products_chromadb(query_embedding=query_embedding, k=3)
formatted_response = format_response_with_openai(productos_relacionados, gpt_client)

# Imprime el resultado formateado
print("\nRespuestas formateadas:")
for idx, response in enumerate(formatted_response, 1):
    print(f"\nProducto {idx}:")
    print(response)
    print("-" * 50)


Respuestas formateadas:

Producto 1:
El Aceite de Soya BASSO en botella de 1L es un aceite vegetal ideal para preparar frituras. No absorbe fácilmente, lo que lo hace perfecto para obtener una textura crujiente en tus platillos. Es una opción versátil para diversas recetas, desde arroces hasta papitas fritas.

Este producto es una excelente alternativa en la cocina debido a su versatilidad y capacidad para realzar el sabor de tus comidas. Además, su presentación en botella de 1 litro lo hace conveniente y fácil de almacenar.

Puedes encontrar el Aceite de Soya BASSO en PlazaVea a un precio de S/ 12.2. Pruébalo en tus platillos favoritos y disfruta de su calidad y beneficios en tus recetas diarias.
--------------------------------------------------

Producto 2:
El Aceite de Maní BASSO viene en una botella de 1 litro y es perfecto para dar ese toque especial a tus ensaladas o guisos. Es un aceite especial que contiene nutrientes adicionales que pueden beneficiar tu organismo. Es ideal p